<a href="https://colab.research.google.com/github/AlexH20/Supervised-ML-sentiment-measures/blob/main/FinBERT_with_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip3 install torch
!pip3 install transformers -i https://pypi.python.org/simple

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.python.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import numpy as np
import torch
from transformers import BertTokenizer
from torch import nn
from transformers import BertModel
from torch.optim import Adam
from tqdm import tqdm
from google.colab import drive 
import pandas as pd
from sklearn.model_selection import train_test_split
import csv

drive.mount("/content/gdrive")

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = df['AR_dummy'].tolist()
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['Text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.1):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('yiyanghkust/finbert-pretrain')

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)

        return pooled_output.tolist()

def train(model, train_data):

    train = Dataset(train_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=8, shuffle=True)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

            model = model.cuda()

    frames = []

    for train_input, train_label in tqdm(train_dataloader):

        train_label = train_label.to(device)
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)

        df = pd.DataFrame(model(input_id, mask))

        frames.append(df)

    df_final = pd.concat(frames)

    return df_final


def split_months(dt):
    return [dt[dt["ordered_month"] == y] for y in dt["ordered_month"].unique()]

data = pd.read_csv("gdrive/My Drive/Thesis/processed data/CAR_regression/datasets_final/data_whole_woScAR.csv", index_col = False)

data_onlytext = data[data["word_count"] != 0]
data_onlytext["Date"] = pd.to_datetime(data_onlytext["Date"])
data_onlytext["Year"] = [x.year for x in data_onlytext["Date"]]
data_onlytext["Month"] = [x.month for x in data_onlytext["Date"]]
data_onlytext["ordered_month"] = [((x[1]["Year"]-2015)*12 + x[1]["Month"]) for x in data_onlytext.iterrows()]


tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')
  
data_splt_months = split_months(data_onlytext)

i = -1

np.random.seed(9000)
for _, month in enumerate(data_splt_months):

        i += 1

        data_train = pd.concat([data_splt_months[i], data_splt_months[i+1], data_splt_months[i+2]])
        print(data_train)
        data_test = data_splt_months[i+3]

        model = BertClassifier()
        X_train_hidden = train(model, data_train)
        X_test_hidden = train(model, data_test)

        y_train = data_train["AR"]

        
        
"""
        if i+1 < len(data_splt_months):

            data_splt_months[i+3]["AR_BERT"] = pred

            with open("gdrive/My Drive/Thesis/processed data/CAR_regression/BERT_sentiment_dummy_R/" + str(i+1) + ".csv", "w") as csv_file:
                  
                  writer = csv.writer(csv_file)
                  writer.writerow(
                      ["Date", "Ticker", "Nasdaq", "Turnover", "Size", "BTM", "pref_alpha", "CAR", "Text", "AR", "AR_dummy", "AR_BERT"])
                  for index, row in data_splt_months[i+3].iterrows():
                      writer.writerow([row["Date"], row["Ticker"], row["Nasdaq"], row["Turnover"], row["Size"], row["BTM"], row["pref_alpha"], row["CAR"], row["Text"], row["AR"], row["AR_dummy"], row["AR_BERT"]])

"""








Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

             Date Ticker  Nasdaq  Turnover          Size       BTM  \
0      2015-01-02   AAPL       1  1.336802  6.370024e+08  0.187370   
1      2015-01-05   AAPL       1  1.344416  6.190077e+08  0.192817   
2      2015-01-06   AAPL       1  1.347419  6.190077e+08  0.192817   
7      2015-01-13   AAPL       1  1.376746  6.363537e+08  0.187561   
8      2015-01-14   AAPL       1  1.384635  6.420037e+08  0.185910   
...           ...    ...     ...       ...           ...       ...   
119559 2015-03-16    GLW       0  1.066449  3.014349e+07  0.623285   
120809 2015-03-04    ADP       1  0.631143  4.214598e+07  0.114092   
120824 2015-03-25    ADP       1  0.662422  4.132159e+07  0.116368   
124599 2015-03-26    ACN       0  0.724509  5.540726e+07  0.110703   
124601 2015-03-30    ACN       0  0.724411  5.930794e+07  0.103422   

        pref_alpha       CAR  \
0         0.001312 -0.018882   
1         0.001142 -0.000292   
2         0.000912  0.012774   
7         0.000836  0.012517   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT e

         0         1         2         3         4         5         6    \
0   0.999113  0.146889 -0.991251  0.709346 -0.780556  0.203437  0.254314   
1   0.999486  0.306045 -0.978605  0.832530 -0.764886  0.224046  0.180493   
2   0.999542  0.039582 -0.967300  0.714708 -0.884678  0.266150  0.614709   
3   0.999250  0.179671 -0.994733  0.582414 -0.454200  0.369296 -0.845250   
4   0.999688  0.176820 -0.988194  0.930342 -0.902581  0.188742  0.297039   
..       ...       ...       ...       ...       ...       ...       ...   
0   0.998588  0.098317 -0.946693  0.290088 -0.663621  0.296852  0.189857   
1   0.999568  0.091808 -0.978617  0.952216 -0.937682  0.233454  0.484442   
2   0.993600  0.101305 -0.998607  0.655180 -0.257875  0.201596 -0.779295   
3   0.999332  0.210643 -0.944445  0.733916 -0.677465  0.434572 -0.200616   
4   0.999270  0.068906 -0.995208  0.844726 -0.470104  0.405952 -0.277172   

         7         8         9    ...       758       759       760       761  \
0  -0.

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 28%|██▊       | 34/122 [00:03<00:08, 10.26it/s]


KeyboardInterrupt: ignored